## Test Notebook

In [1]:
import sys
from rdkit import Chem
from rdkit.Chem import AllChem
import rdkit.Chem.MolStandardize
import rdkit.Chem.MolStandardize.rdMolStandardize
import pandas as pd
import numpy as np
from pyteomics import mgf
import matplotlib.pyplot as plt
from data_utils import *
import operator
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import optimizers
import tensorflow as tf

RDKit WARNING: [20:45:43] Enabling RDKit 2019.09.1 jupyter extensions
Using TensorFlow backend.


In [2]:
## Check GPU available
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.config.experimental.list_physical_devices('GPU'))
tf.debugging.set_log_device_placement(True)

Num GPUs Available:  0
[]


In [ ]:
unique_data = fetch_data("unique")

In [ ]:
## find most common large peak
# peaks = count_peaks(unique_data, 0.9)
# max_peak = max(peaks.items(), key=operator.itemgetter(1))[0]
max_peak = 121

##### predict only bin 121
x_data = []
y_data = []

for i, mol in unique_data.iterrows():
    
    if i % 500 == 0:
        sys.stdout.write("Binning: %d   \r" % (i) )
        sys.stdout.flush()
        
    x_data.append(mol['fingerprint'])
    y_data.append(mol['normed_binned'][max_peak])
    
    
split = int(0.8 * len(x_data))
    
x_train = np.array(x_data[:split])
y_train = np.array(y_data[:split])

x_test = np.array(x_data[split:])
y_test = np.array(y_data[split:])

len(x_data)

In [ ]:
### First working MLP on dense data

# model = Sequential()
# model.add(Dense(units=1024, activation='sigmoid', input_shape=(2048,)))
# model.add(Dense(units=64, activation='relu'))
# model.add(Dense(1, activation='linear'))
# sgd = optimizers.SGD(lr=0.01, nesterov=True);
# model.compile(loss='mean_absolute_error', optimizer=sgd)


### First working MLP on sparse data

# Lrelu = keras.layers.LeakyReLU(alpha=0.3)
# model = Sequential()
# model.add(Dense(units=2048, activation=Lrelu, input_shape=(2048,)))
# model.add(Dense(units=128, activation=Lrelu))
# model.add(Dense(units=1, activation=Lrelu))
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'mae', 'mape', 'cosine'])

In [ ]:
model = Sequential()
model.add(Dense(units=1024, activation='relu', input_shape=(2048,)))
model.add(Dense(units=512, activation='relu'))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=1, activation='relu'))

model.compile(loss='mean_absolute_error', optimizer='sgd', metrics=['mse'])

In [ ]:
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

In [ ]:
results = model.evaluate(x_test, y_test, batch_size=32, verbose=1)
labels = model.metrics_names

for i in range(len(results)):
    print(labels[i], results[i])
    
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
preds = model.predict(np.array(x_test))
print(np.all(preds == 0.0))
for i, y in enumerate(preds):
    print(y, " => ", y_test[i])

In [ ]:
x2_data = []
y2_data = []

for i, mol in unique_data.iterrows():
    
    if i % 500 == 0:
        sys.stdout.write("Collecting: %d   \r" % (i) )
        sys.stdout.flush()
        
    x2_data.append(mol['fingerprint'])
    y2_data.append(mol['normed_binned'][50:100])
    
    
split = int(0.8 * len(x2_data))
    
x2_train = np.array(x2_data[:split])
y2_train = np.array(y2_data[:split])

x2_test = np.array(x2_data[split:])
y2_test = np.array(y2_data[split:])

len(x2_data)

In [ ]:
# model2 = Sequential()
# model2.add(Dense(units=2048, activation='relu', input_shape=(2048,)))
# #model2.add(Dropout(0.1))
# #model2.add(Dense(units=2000, activation='relu'))
# #model2.add(Dropout(0.1))
# #model2.add(Dense(units=2000, activation='relu'))
# model2.add(Dropout(0.1))
# model2.add(Dense(units=2000, activation='relu'))
# model2.add(Dropout(0.1))
# model2.add(Dense(units=2000, activation='relu'))
# model2.add(Dropout(0.1))
# model2.add(Dense(units=50, activation='relu'))
    
adam = keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False)

params_array = [
    #{'optimizer': 'sgd', 'loss': 'mean_absolute_percentage_error'},
    {'optimizer': adam, 'loss': 'mean_absolute_percentage_error'},
    #{'optimizer': 'rmsprop', 'loss': 'mean_absolute_percentage_error'},
    #{'optimizer': 'adamax', 'loss': 'mean_absolute_percentage_error'},
]

models = []
histories = []

for i, params in enumerate(params_array):
    print(params['optimizer'], params['loss'])
    
    models.append(Sequential())
    models[i].add(Dense(units=2048, activation='relu', input_shape=(2048,)))
    #models2.add(Dropout(0.1))
    #models2.add(Dense(units=2000, activation='relu'))
    #models2.add(Dropout(0.1))
    #models2.add(Dense(units=2000, activation='relu'))
    models[i].add(Dropout(0.1))
    models[i].add(Dense(units=2000, activation='relu'))
    models[i].add(Dropout(0.1))
    models[i].add(Dense(units=2000, activation='relu'))
    models[i].add(Dropout(0.1))
    models[i].add(Dense(units=50, activation='relu'))
    
    models[i].compile(loss='mean_absolute_percentage_error', optimizer=adam)
    histories.append(models[i].fit(x2_train, y2_train, epochs=5, batch_size=32, validation_split=0.2, verbose=1))

In [ ]:
results2 = model2.evaluate(x2_test, y2_test, batch_size=32, verbose=1)
labels2 = model2.metrics_names

for i in range(len(results2)):
    print(labels2[i], results2[i])
    
# Plot training & validation loss values
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.title('Model loss (MSE)')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
base = 3
show_spectrum(unique_data.iloc[base])

In [ ]:
# file = open("duplicates.txt", "r")
# f1 = file.readlines()
# len(f1)

# distinct = []
# unique_pairs = []
# for string in f1:
#     matched = string.split(", ")
#     x = int(matched[0])
#     y = int(matched[1])
    
#     pair = (x, y)
#     reverse = (y, x)
    
#     if x not in distinct:
#         distinct.append(x)
        
#     if pair not in unique_pairs and reverse not in unique_pairs:
#         unique_pairs.append(pair)

        
# print(len(distinct))
# print(len(unique_pairs))

## combinations w/out repitition =   n! / r!(n-r)! == 33936441
## 2326 / 33936441 * 100 = 0.006853989%

In [ ]:
compare_spectrum(unique_data.iloc[0], unique_data.iloc[775])

In [ ]:
predictions50 = models[0].predict(x2_test)

for i in range(len(y2_test)):
    if not np.all(predictions50[i] == 0):        
        compare_bins(y2_test[i], predictions50[i])

In [ ]:
n = 2

xn_data = []
yn_data = []

for i, mol in unique_data.iterrows():
    
    if i % 500 == 0:
        sys.stdout.write("Collecting: %d   \r" % (i) )
        sys.stdout.flush()
        
    xn_data.append(mol['fingerprint'])
    yn_data.append(mol['normed_binned'][50:100])
    
    
split = int(0.8 * len(xn_data))
    
xn_train = np.array(xn_data[:split])
yn_train = np.array(yn_data[:split])

xn_test = np.array(xn_data[split:])
yn_test = np.array(yn_data[split:])

print(len(x2_data))


model2 = Sequential()
model2.add(Dense(units=2048, activation='relu', input_shape=(2048,)))
model2.add(Dropout(0.1))
model2.add(Dense(units=2000, activation='relu'))
model2.add(Dropout(0.1))
model2.add(Dense(units=2000, activation='relu'))
model2.add(Dropout(0.1))
model2.add(Dense(units=2000, activation='relu'))
model2.add(Dropout(0.1))
model2.add(Dense(units=2000, activation='relu'))
model2.add(Dropout(0.1))
model2.add(Dense(units=2000, activation='relu'))
model2.add(Dropout(0.1))
model2.add(Dense(units=2000, activation='relu'))
model2.add(Dropout(0.1))
model2.add(Dense(units=50, activation='relu'))

model2.compile(loss='mean_absolute_percentage_error', optimizer=adam)
history2 = model2.fit(x2_train, y2_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

In [ ]:
bin_counts = np.zeros(3000)

for i, mol in unique_data.iterrows():
    for j, val in enumerate(mol['normed_binned']):
        if val > 0.01:
            bin_counts[j] = bin_counts[j] + 1

In [ ]:
bin_counts

In [ ]:
fig = plt.figure(figsize=(14,10))
ax = fig.add_subplot(1,1,1)
ax.plot(bin_counts[:200])

In [ ]:
i = 0
nonzero = False
while not nonzero:
    if bin_counts[i] > 0:
        print(bin_counts[i])
        nonzero = True
    i += 1
    
print(i)